# データ読込

In [1]:
import pandas as pd
from logic.factory_manage.sql import load_data_from_sqlite
from utils.get_holydays import get_japanese_holidays
from logic.factory_manage.predict_model_v3 import generate_features, train_and_predict

# CSVファイル読み込み（パスは適宜変更）
path = "/work/app/data/factory_manage/weight_data.db"
df = load_data_from_sqlite()
df["伝票日付"].max()

hol_max = df["伝票日付"].max()
hol_min = df["伝票日付"].min()
print(f"最小日付: {hol_min}, 最大日付: {hol_max}")
holiday = get_japanese_holidays(hol_min, hol_max)


最小日付: 2024-05-01 00:00:00, 最大日付: 2025-06-22 00:00:00


In [2]:
# 品目別合計重量を集計
item_totals = df.groupby("品名")["正味重量"].sum().sort_values(ascending=False)

print(item_totals)


品名
混合廃棄物A          14938790.0
混合廃棄物B           3864960.0
GC 軽鉄･ｽﾁｰﾙ類      2463090.0
選別               1522350.0
木くず              1399580.0
                   ...    
混合廃棄物A(廃ﾌﾟﾗ)          30.0
雑誌                    30.0
選別（水銀灯）               30.0
ﾀｲﾔ                   20.0
混合廃棄物(羽毛)             10.0
Name: 正味重量, Length: 156, dtype: float64


In [3]:
df_reserve = pd.read_csv(
    "/work/app/data/input/yoyaku_data.csv")
df_reserve["予約日"] = pd.to_datetime(df_reserve["予約日"])
df_reserve = df_reserve[df_reserve["予約日"] >= hol_min]
df_reserve = df_reserve[df_reserve["予約日"] <= hol_max]
df_reserve

,予約日,予約得意先名,固定客
0,2025-03-02,泉土木,True
1,2025-03-02,ASMU,False
2,2025-03-02,青木サービス,False
3,2025-03-02,オネストワン,False
4,2025-03-02,おかたずけ田中くん,False
...,...,...,...
4743,2025-05-31,山口興業,True
4744,2025-05-31,谷津商会,True
4745,2025-05-31,吉田機電,False
4746,2025-05-31,ヨシモリ,True


In [4]:
print(df_reserve.columns)

Index(['予約日', '予約得意先名', '固定客'], dtype='object')


## 特徴量生成

In [5]:
# from logic.factory_manage.predict_model_v4_3 import full_walkforward_pipeline

# pred_df = full_walkforward_pipeline(
#     df,
#     holidays=holiday,
#     df_reserve=df_reserve
# )


In [6]:
from logic.factory_manage.predict_model_v4_4 import full_walkforward_pipeline

target_items = ["混合廃棄物A", "混合廃棄物B", "GC 軽鉄･ｽﾁｰﾙ類", "選別", "木くず"]
pred_df_slim = full_walkforward_pipeline(
    df,
    holidays=holiday,
    target_items=target_items
)


===== 2024-05-10 処理中 =====


    混合廃棄物A 予測値: 51000.0kg
    混合廃棄物B 予測値: 5320.0kg
    GC 軽鉄･ｽﾁｰﾙ類 予測値: 2440.0kg
    選別 予測値: 3010.0kg
    木くず 予測値: 7300.0kg

===== 2024-05-11 処理中 =====
    混合廃棄物A 予測値: 45620.9kg
    混合廃棄物B 予測値: 9250.8kg
    GC 軽鉄･ｽﾁｰﾙ類 予測値: 2180.8kg
    選別 予測値: 1890.2kg
    木くず 予測値: 9842.5kg

===== 2024-05-12 処理中 =====
    混合廃棄物A 予測値: 42893.8kg
    混合廃棄物B 予測値: 14878.5kg
    GC 軽鉄･ｽﾁｰﾙ類 予測値: 5591.8kg
    選別 予測値: 2819.8kg
    木くず 予測値: 7050.9kg

===== 2024-05-13 処理中 =====
    混合廃棄物A 予測値: 10054.7kg
    混合廃棄物B 予測値: 0.0kg
    GC 軽鉄･ｽﾁｰﾙ類 予測値: 1452.9kg
    選別 予測値: 567.3kg
    木くず 予測値: 2256.0kg

===== 2024-05-14 処理中 =====
    混合廃棄物A 予測値: 49148.9kg
    混合廃棄物B 予測値: 13452.0kg
    GC 軽鉄･ｽﾁｰﾙ類 予測値: 2727.7kg
    選別 予測値: 3931.9kg
    木くず 予測値: 4201.1kg

===== 2024-05-15 処理中 =====
    混合廃棄物A 予測値: 59463.4kg
    混合廃棄物B 予測値: 12951.0kg
    GC 軽鉄･ｽﾁｰﾙ類 予測値: 1182.4kg
    選別 予測値: 3419.3kg
    木くず 予測値: 5728.9kg

===== 2024-05-16 処理中 =====
    混合廃棄物A 予測値: 47905.8kg
    混合廃棄物B 予測値: 14509.8kg
    GC 軽鉄･ｽﾁｰﾙ類 予測値: 699.8kg
    選別 予